In [1]:
%conda install pandas

Channels:
 - defaults
Platform: linux-aarch64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.3.0
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /workspaces/parallel-corpora2/.conda

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |         openblas           8 KB
    bottleneck-1.4.2           |  py312h0d09708_0         136 KB
    libgfortran-ng-11.2.0      |       h6e398d7_1          20 KB
    libgfortran5-11.2.0        |       h1234567_1         1.3 MB
    libopenblas-0.3.21         |       hc2e42e2_0         4.4 MB
    numexpr-2.10.1             |  py312hd351863_0         201 KB
    numpy-2.0.1                |  py312hdb1dca2_1          11 KB
    numpy-base-2.0.

In [11]:
!gcloud storage cp data/ocr/src/astangasangraha/astangasangraha.pdf  gs://iiith-ska-satamt/src/astangasangraha/astangasangraha.pdf

Copying file://data/ocr/src/astangasangraha/astangasangraha.pdf to gs://iiith-ska-satamt/src/astangasangraha/astangasangraha.pdf
  Completed files 1/1 | 62.2MiB/62.2MiB | 406.4kiB/s                           

Average throughput: 644.7kiB/s


In [ ]:
!gcloud ml vision detect-text-pdf gs://iiith-ska-satamt/src/astangasangraha/astangasangraha.pdf  gs://iiith-ska-satamt/ast_vol1_output/

In [ ]:
# prompt: read this file gcv_test_sample1_outputoutput-1-to-20.json
# read 'responses' field. for each entry in reponses
# get fullTextAnnotation.text, context.uri, context.pageNumber
# display as csv table

import os
import json
import pandas as pd

def natural_key(s):
    return [int(text) if text.isdigit() else text for text in re.split(r'(\d+)', s)]

def process_json_file(filepath):
    try:
        with open(filepath, 'r') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {filepath}")
        return None

    responses = data.get('responses', [])
    if not responses:
        print("Warning: 'responses' field not found or empty in JSON data.")
        return None

    extracted_data = []
    for response in responses:
        full_text_annotation = response.get('fullTextAnnotation', {})
        text = full_text_annotation.get('text', '')
        context = response.get('context', {})
        uri = context.get('uri', '')
        page_number = context.get('pageNumber', '')
        extracted_data.append({'text': text, 'uri': uri, 'page_number': page_number})

    return extracted_data

    # df = pd.DataFrame(extracted_data)
    # return df

# For all the files in the data/ast_vol1_output folder 
# and process them
src_dir = 'data/ast_vol1_output'

# Sort files based on natural order
files = sorted([f for f in os.listdir(src_dir) if f.endswith(".json")], key=natural_key)

with open('data/ast_vol1_ouput.md', 'w') as md_file:
    md_file.write("# ast_vol1_ouput \n")
    md_file.write("\n---\n")

    for filename in files:
        if filename.endswith('.json'):
            filepath = os.path.join(src_dir, filename)
            data = process_json_file(filepath)
            if data is None:
                print(f"Failed to process file: {filename}")
                continue

            print(f"Processing file: {filename}")

            for entry in data:
                page_number = entry.get('page_number')
                text = entry.get('text')
                
                if not text:
                    print(f"Warning: 'text' field is empty in entry from {filename} on page {page_number}.")
                    continue

                md_file.write(f"\n## Page: {page_number}\n")
                md_file.write("```")
                md_file.write(f"\n{text}\n")
                md_file.write("```")
                md_file.write("\n---\n")



Processing file: output-1-to-20.json
Processing file: output-21-to-40.json
Processing file: output-41-to-60.json
Processing file: output-61-to-80.json
Processing file: output-81-to-100.json
Processing file: output-101-to-120.json
Processing file: output-121-to-140.json
Processing file: output-141-to-160.json
Processing file: output-161-to-180.json
Processing file: output-181-to-200.json
Processing file: output-201-to-220.json
Processing file: output-221-to-240.json
Processing file: output-241-to-260.json
Processing file: output-261-to-280.json
Processing file: output-281-to-300.json
Processing file: output-301-to-320.json
Processing file: output-321-to-340.json
Processing file: output-341-to-360.json
Processing file: output-361-to-380.json
Processing file: output-381-to-400.json
Processing file: output-401-to-420.json
Processing file: output-421-to-440.json
Processing file: output-441-to-460.json
Processing file: output-461-to-480.json
Processing file: output-481-to-500.json
Processing